# Extracción, transformación y carga (ETL)

En la parte superior cargaremos los paquetes

In [2]:
# Cargar paquetes
import pandas as pd
import numpy as np

* Carguemos los datos recopilados para realizar los análisis

In [3]:
# Cargar los datos
datos_recopilados = pd.read_excel('C:/Users/cdeor/OneDrive/Documentos/Python/jb_aet_pm10_25/docs/_data/data_1.xlsx')

# Renombrar las columnas
datos_recopilados.rename(columns={
    'Estacion': 'Estación',
    'Fecha inicial': 'Fecha',
    'PARÁMETRO': 'Parámetro',
    'CONCENTRACIÓN mg/m3': 'Concentración_mg/m3'
}, inplace=True)

datos_recopilados.head(5)

,Estación,Fecha,Parámetro,Concentración_mg/m3
0,LA GLORIA,2022-12-21 00:00:00,PM 10,40.69
1,LA GLORIA,2022-12-20 00:00:00,PM 10,41.01
2,LA GLORIA,2022-12-19 00:00:00,PM 10,40.94
3,LA GLORIA,2022-12-18,PM 10,39.87
4,LA GLORIA,2022-12-17,PM 10,38.79


* Ajustemos la fecha como `datetime`:

In [4]:
# Convertir la columna "Fecha" a formato datetime
datos_recopilados['Fecha'] = pd.to_datetime(datos_recopilados['Fecha'], dayfirst=True)

# Mostrar los datos para verificar
datos_recopilados.head(5)

,Estación,Fecha,Parámetro,Concentración_mg/m3
0,LA GLORIA,2022-12-21,PM 10,40.69
1,LA GLORIA,2022-12-20,PM 10,41.01
2,LA GLORIA,2022-12-19,PM 10,40.94
3,LA GLORIA,2022-12-18,PM 10,39.87
4,LA GLORIA,2022-12-17,PM 10,38.79


* Información de los datos de recopilación

In [5]:
datos_recopilados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Estación             1352 non-null   object        
 1   Fecha                1352 non-null   datetime64[ns]
 2   Parámetro            1352 non-null   object        
 3   Concentración_mg/m3  1352 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 42.4+ KB


* Carguemos los datos de parámetros

In [6]:
# Cargar los datos
datos_parametros = pd.read_excel('C:/Users/cdeor/OneDrive/Documentos/Python/jb_aet_pm10_25/docs/_data/parametros.xlsx')

# Eliminar las columnas especificadas
datos_parametros = datos_parametros.drop(columns=['Unnamed: 1', 'Unnamed: 3'])

# Renombrar las columnas
datos_parametros.rename(columns={
    'FECHA ': 'Fecha',
    'VALOR': 'Valor',
    'UNIDAD': 'Unidad',
    'PARAMETRO': 'Parametros_climaticos'
}, inplace=True)

# Convertir la columna "Fecha" a formato datetime
datos_parametros['Fecha'] = pd.to_datetime(datos_parametros['Fecha'], format='%Y-%m-%d %H:%M', errors='coerce')

datos_parametros.head()

,Fecha,Valor,Unidad,Parametros_climaticos
0,2020-06-17,249.907970,Grados (°),DIR VIENTO
1,2020-06-18,106.967656,Grados (°),DIR VIENTO
2,2020-06-19,352.443983,Grados (°),DIR VIENTO
3,2020-06-20,161.221664,Grados (°),DIR VIENTO
4,2020-06-21,192.649780,Grados (°),DIR VIENTO


* Información de los datos de parámetros

In [7]:
datos_parametros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5121 entries, 0 to 5120
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  5121 non-null   datetime64[ns]
 1   Valor                  5121 non-null   float64       
 2   Unidad                 5121 non-null   object        
 3   Parametros_climaticos  5121 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 160.2+ KB


In [8]:
# Unir los DataFrames por la columna "Fecha" usando un merge (left join para mantener todos los datos de df_parametros)
df = pd.merge(datos_parametros, datos_recopilados, on='Fecha', how='left')
df.info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8892 entries, 0 to 8891
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  8892 non-null   datetime64[ns]
 1   Valor                  8892 non-null   float64       
 2   Unidad                 8892 non-null   object        
 3   Parametros_climaticos  8892 non-null   object        
 4   Estación               4751 non-null   object        
 5   Parámetro              4751 non-null   object        
 6   Concentración_mg/m3    4751 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 486.4+ KB


In [10]:
df.head(20)

,Fecha,Valor,Unidad,Parametros_climaticos,Estación,Parámetro,Concentración_mg/m3
0,2020-06-17,249.907970,Grados (°),DIR VIENTO,NaN,NaN,NaN
1,2020-06-18,106.967656,Grados (°),DIR VIENTO,NaN,NaN,NaN
2,2020-06-19,352.443983,Grados (°),DIR VIENTO,NaN,NaN,NaN
3,2020-06-20,161.221664,Grados (°),DIR VIENTO,NaN,NaN,NaN
4,2020-06-21,192.649780,Grados (°),DIR VIENTO,NaN,NaN,NaN
5,2020-06-22,154.702942,Grados (°),DIR VIENTO,NaN,NaN,NaN
6,2020-06-23,131.504356,Grados (°),DIR VIENTO,NaN,NaN,NaN
7,2020-06-24,25.817766,Grados (°),DIR VIENTO,NaN,NaN,NaN
8,2020-06-25,8.985866,Grados (°),DIR VIENTO,NaN,NaN,NaN
9,2020-06-26,348.034911,Grados (°),DIR VIENTO,NaN,NaN,NaN


In [ ]:
# Unir los DataFrames por la columna "Fecha" usando un merge (left join para mantener todos los datos de df_parametros)
df_2 = pd.merge(datos_parametros, datos_recopilados, on='Fecha', how='right')
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4791 entries, 0 to 4790
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  4791 non-null   datetime64[ns]
 1   Valor                  4751 non-null   float64       
 2   Unidad                 4751 non-null   object        
 3   Parametros_climaticos  4751 non-null   object        
 4   Estación               4791 non-null   object        
 5   Parámetro              4791 non-null   object        
 6   Concentración_mg/m3    4791 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 262.1+ KB


In [11]:
df_2.head(20)

,Fecha,Valor,Unidad,Parametros_climaticos,Estación,Parámetro,Concentración_mg/m3
0,2022-12-21,NaN,NaN,NaN,LA GLORIA,PM 10,40.69
1,2022-12-20,NaN,NaN,NaN,LA GLORIA,PM 10,41.01
2,2022-12-19,NaN,NaN,NaN,LA GLORIA,PM 10,40.94
3,2022-12-18,NaN,NaN,NaN,LA GLORIA,PM 10,39.87
4,2022-12-17,NaN,NaN,NaN,LA GLORIA,PM 10,38.79
5,2022-12-16,NaN,NaN,NaN,LA GLORIA,PM 10,39.36
6,2022-12-15,NaN,NaN,NaN,LA GLORIA,PM 10,42.59
7,2022-12-14,NaN,NaN,NaN,LA GLORIA,PM 10,43.79
8,2022-12-13,NaN,NaN,NaN,LA GLORIA,PM 10,40.12
9,2022-12-12,NaN,NaN,NaN,LA GLORIA,PM 10,44.87
